# Learning: algorithms, objectives, and assumptions
(c) Deniz Yuret 2019

In this notebook we will analyze three classic learning algorithms.
* **Perceptron:** ([Rosenblatt, 1957](https://en.wikipedia.org/wiki/Perceptron)) a neuron model trained with a simple algorithm that updates model weights using the input when the prediction is wrong.
* **Adaline:** ([Widrow and Hoff, 1960](https://en.wikipedia.org/wiki/ADALINE)) a neuron model trained with a simple algorithm that updates model weights using the error multiplied by the input (aka least mean square (LMS), delta learning rule, or the Widrow-Hoff rule).
* **Softmax classification:** ([Cox, 1958](https://en.wikipedia.org/wiki/Multinomial_logistic_regression)) a multiclass generalization of the logistic regression model from statistics (aka multinomial LR, softmax regression, maxent classifier etc.).

We will look at these learners from three different perspectives:
* **Algorithm:** First we ask only **how** the learner works, i.e. how it changes after observing each example.
* **Objectives:** Next we ask **what** objective guides the algorithm, whether it is optimizing a particular objective function, and whether we can use a generic *optimization algorithm* instead.
* **Assumptions:** Finally we ask **why** we think this algorithm makes sense, what prior assumptions does this imply and whether we can use *probabilistic inference* for optimal learning.

In [1]:
using Knet, Plots, Statistics, LinearAlgebra, Random
Base.argmax(a::KnetArray) = argmax(Array(a))
Base.argmax(a::AutoGrad.Value) = argmax(value(a))
ENV["COLUMNS"] = 72

72

### Data

In [2]:
include(Knet.dir("data/mnist.jl"))
xtrn, ytrn, xtst, ytst = mnist()
ARRAY = Array{Float32}
xtrn, xtst = ARRAY(mat(xtrn)), ARRAY(mat(xtst))
onehot(y) = (m=ARRAY(zeros(maximum(y),length(y))); for i in 1:length(y); m[y[i],i]=1; end; m)
ytrn, ytst = onehot(ytrn), onehot(ytst);

┌ Info: Loading MNIST...
└ @ Main /kuacc/users/dyuret/.julia/packages/Knet/LjPts/data/mnist.jl:33


In [3]:
println.(summary.((xtrn, ytrn, xtst, ytst)));

784×60000 Array{Float32,2}
10×60000 Array{Float32,2}
784×10000 Array{Float32,2}
10×10000 Array{Float32,2}


In [4]:
NTRN,NTST,XDIM,YDIM = size(xtrn,2), size(xtst,2), size(xtrn,1), size(ytrn,1)

(60000, 10000, 784, 10)

### Prediction

In [5]:
# Model weights
w = ARRAY(randn(YDIM,XDIM))

10×784 Array{Float32,2}:
 -0.723526    0.984467   1.53663   …  -1.02691    0.827406 
 -1.23713    -0.130535  -0.426584     -2.98049    0.236315 
 -1.01081     0.599193   0.39188      -0.254158  -0.0685053
  1.95004     1.12775   -0.270512      1.50776    0.38962  
 -1.65777    -1.30566    1.49927      -0.288618   1.3921   
  0.247336    0.37563    1.53998   …  -0.227825   0.81309  
 -0.0449449  -0.775154   0.535788      0.749737  -0.534139 
  0.254892   -0.444621   0.640151      0.130355   0.64397  
 -1.21859    -0.588326   1.21659       0.408257   0.723103 
  0.557634    0.340867   0.159563      0.540516   0.63455  

In [6]:
# Class scores
w * xtrn

10×60000 Array{Float32,2}:
   4.96523  -7.80919  -24.0313    -0.364504  …  -10.8263    -9.00563 
  13.2433    3.92816    1.89772   -9.89675       -4.45021  -15.0672  
 -12.9268    2.97658   -7.45577  -16.8185        -3.89602  -10.5781  
   2.05194  -2.72613   -8.16912    0.715291       3.48574    4.46081 
  -6.13742  -5.74059    1.41782  -17.024         -1.11403    0.278054
   9.04555   4.09188    3.02878    7.18583   …    9.06085   -6.22643 
  12.2155    9.27237    4.17152   -6.02936        4.86192   -2.0607  
  -7.35357  -2.72163   -4.7295     3.90331        3.58258    1.90419 
  -7.79106  -3.47143   -6.35381    2.64577       -7.66081  -19.9355  
  -6.70359  -7.68111   -6.4374   -11.2293         3.27617   -9.95783 

In [7]:
# Predictions
[ argmax(w * xtrn[:,i]) for i in 1:NTRN ]'

1×60000 Adjoint{Int64,Array{Int64,1}}:
 2  7  7  6  2  6  4  7  6  7  7  …  4  4  6  4  7  6  7  2  7  6  4

In [8]:
# Correct answers
[ argmax(ytrn[:,i]) for i in 1:NTRN ]'

1×60000 Adjoint{Int64,Array{Int64,1}}:
 5  10  4  1  9  2  1  3  1  4  3  …  8  9  2  9  5  1  8  3  5  6  8

In [9]:
# Accuracy
acc(w,x,y) = mean(argmax(w * x, dims=1) .== argmax(y, dims=1))
acc(w,xtrn,ytrn), acc(w,xtst,ytst)

(0.0833, 0.0746)

## Algorithms

In [10]:
# Training loop
function train(algo,x,y,T=2^20)
    w = ARRAY(zeros(size(y,1),size(x,1)))
    nexamples = size(x,2)
    nextprint = 1
    for t = 1:T
        i = rand(1:nexamples)
        algo(w, x[:,i], y[:,i])  # <== this is where w is updated
        if t == nextprint
            println((iter=t, accuracy=acc(w,x,y), wnorm=norm(w)))
            nextprint = min(2t,T)
        end
    end
    w
end

train (generic function with 2 methods)

### Perceptron

In [11]:
function perceptron(w,x,y)
    guess = argmax(w * x)
    class = argmax(y)
    if guess != class
        w[class,:] .+= x
        w[guess,:] .-= x
    end
end

perceptron (generic function with 1 method)

In [12]:
# (iter = 1048576, accuracy = 0.8950333333333333, wnorm = 1321.2463f0) in 7 secs
@time wperceptron = train(perceptron,xtrn,ytrn);

(iter = 1, accuracy = 0.09871666666666666, wnorm = 16.555326f0)
(iter = 2, accuracy = 0.12575, wnorm = 19.439667f0)
(iter = 4, accuracy = 0.13355, wnorm = 20.399765f0)
(iter = 8, accuracy = 0.20878333333333332, wnorm = 25.85245f0)
(iter = 16, accuracy = 0.11148333333333334, wnorm = 38.57418f0)
(iter = 32, accuracy = 0.32356666666666667, wnorm = 48.58623f0)
(iter = 64, accuracy = 0.2728, wnorm = 66.928535f0)
(iter = 128, accuracy = 0.60465, wnorm = 89.28772f0)
(iter = 256, accuracy = 0.6815666666666667, wnorm = 114.6649f0)
(iter = 512, accuracy = 0.72925, wnorm = 141.36696f0)
(iter = 1024, accuracy = 0.6892333333333334, wnorm = 177.80821f0)
(iter = 2048, accuracy = 0.7991, wnorm = 229.15755f0)
(iter = 4096, accuracy = 0.8287333333333333, wnorm = 280.51797f0)
(iter = 8192, accuracy = 0.8581833333333333, wnorm = 337.9238f0)
(iter = 16384, accuracy = 0.8295, wnorm = 409.2436f0)
(iter = 32768, accuracy = 0.81145, wnorm = 485.44604f0)
(iter = 65536, accuracy = 0.8569333333333333, wnorm = 588

### Adaline

In [13]:
function adaline(w,x,y; lr=0.0001)
    error = w * x - y
    w .-= lr * error * x'
end

adaline (generic function with 1 method)

In [14]:
# (iter = 1048576, accuracy = 0.8523, wnorm = 1.2907721f0) in 31 secs with lr=0.0001
@time wadaline = train(adaline,xtrn,ytrn);

(iter = 1, accuracy = 0.09751666666666667, wnorm = 0.0009055087f0)
(iter = 2, accuracy = 0.15613333333333335, wnorm = 0.001271113f0)
(iter = 4, accuracy = 0.175, wnorm = 0.0019275063f0)
(iter = 8, accuracy = 0.18321666666666667, wnorm = 0.002950374f0)
(iter = 16, accuracy = 0.15383333333333332, wnorm = 0.0043843426f0)
(iter = 32, accuracy = 0.18926666666666667, wnorm = 0.008262319f0)
(iter = 64, accuracy = 0.2615, wnorm = 0.014850053f0)
(iter = 128, accuracy = 0.2799, wnorm = 0.025427587f0)
(iter = 256, accuracy = 0.3463833333333333, wnorm = 0.04235847f0)
(iter = 512, accuracy = 0.6255666666666667, wnorm = 0.06586557f0)
(iter = 1024, accuracy = 0.6563333333333333, wnorm = 0.10303071f0)
(iter = 2048, accuracy = 0.7575333333333333, wnorm = 0.1662105f0)
(iter = 4096, accuracy = 0.7747833333333334, wnorm = 0.25526303f0)
(iter = 8192, accuracy = 0.8031, wnorm = 0.35955703f0)
(iter = 16384, accuracy = 0.82515, wnorm = 0.46123433f0)
(iter = 32768, accuracy = 0.8383166666666667, wnorm = 0.5622

### Softmax classifier

In [15]:
function softmax(w,x,y; lr=0.01)
    probs = exp.(w * x)
    probs ./= sum(probs)
    error = probs - y
    w .-= lr * error * x'
end

softmax (generic function with 1 method)

In [16]:
# (iter = 1048576, accuracy = 0.9242166666666667, wnorm = 26.523603f0) in 32 secs with lr=0.01
@time wsoftmax = train(softmax,xtrn,ytrn);

(iter = 1, accuracy = 0.10441666666666667, wnorm = 0.10300317f0)
(iter = 2, accuracy = 0.16301666666666667, wnorm = 0.13433683f0)
(iter = 4, accuracy = 0.28513333333333335, wnorm = 0.17768018f0)
(iter = 8, accuracy = 0.11805, wnorm = 0.24103987f0)
(iter = 16, accuracy = 0.20396666666666666, wnorm = 0.3208787f0)
(iter = 32, accuracy = 0.18268333333333334, wnorm = 0.50825554f0)
(iter = 64, accuracy = 0.32871666666666666, wnorm = 0.7586994f0)
(iter = 128, accuracy = 0.5855, wnorm = 1.1711375f0)
(iter = 256, accuracy = 0.68105, wnorm = 1.8444412f0)
(iter = 512, accuracy = 0.7839333333333334, wnorm = 2.7081473f0)
(iter = 1024, accuracy = 0.8304833333333334, wnorm = 3.6876163f0)
(iter = 2048, accuracy = 0.8382833333333334, wnorm = 4.728037f0)
(iter = 4096, accuracy = 0.87475, wnorm = 5.873648f0)
(iter = 8192, accuracy = 0.88845, wnorm = 7.210838f0)
(iter = 16384, accuracy = 0.8984833333333333, wnorm = 8.7169895f0)
(iter = 32768, accuracy = 0.9032833333333333, wnorm = 10.475012f0)
(iter = 655

## Objectives

In [17]:
# Training via optimization
function optimize(loss,x,y; lr=0.1, iters=2^20)
    w = Param(ARRAY(zeros(size(y,1),size(x,1))))
    nexamples = size(x,2)
    nextprint = 1
    for t = 1:iters
        i = rand(1:nexamples)
        L = @diff loss(w, x[:,i], y[:,i])
        ∇w = grad(L,w)
        w .-= lr * ∇w
        if t == nextprint
            println((iter=t, accuracy=acc(w,x,y), wnorm=norm(w)))
            nextprint = min(2t,iters)
        end
    end
    w
end

optimize (generic function with 1 method)

### Perceptron minimizes the score difference between the correct class and the prediction

In [18]:
function perceptronloss(w,x,y)
    score = w * x
    guess = argmax(score)
    class = argmax(y)
    score[guess] - score[class]
end

perceptronloss (generic function with 1 method)

In [19]:
# (iter = 1048576, accuracy = 0.8908833333333334, wnorm = 1322.4888f0) in 62 secs with lr=1
@time wperceptron2 = optimize(perceptronloss,xtrn,ytrn,lr=1);

(iter = 1, accuracy = 0.09863333333333334, wnorm = 15.438191f0)
(iter = 2, accuracy = 0.09863333333333334, wnorm = 15.438191f0)
(iter = 4, accuracy = 0.14575, wnorm = 16.696978f0)
(iter = 8, accuracy = 0.13358333333333333, wnorm = 26.538507f0)
(iter = 16, accuracy = 0.25551666666666667, wnorm = 40.80332f0)
(iter = 32, accuracy = 0.38035, wnorm = 49.807693f0)
(iter = 64, accuracy = 0.3633166666666667, wnorm = 71.64613f0)
(iter = 128, accuracy = 0.5086333333333334, wnorm = 86.83524f0)
(iter = 256, accuracy = 0.5217, wnorm = 112.49046f0)
(iter = 512, accuracy = 0.7327833333333333, wnorm = 142.34885f0)
(iter = 1024, accuracy = 0.7181333333333333, wnorm = 184.21814f0)
(iter = 2048, accuracy = 0.7697666666666667, wnorm = 218.95062f0)
(iter = 4096, accuracy = 0.8230666666666666, wnorm = 269.77414f0)
(iter = 8192, accuracy = 0.8184166666666667, wnorm = 332.04178f0)
(iter = 16384, accuracy = 0.8632666666666666, wnorm = 408.29248f0)
(iter = 32768, accuracy = 0.8854833333333333, wnorm = 499.76358

### Adaline minimizes the squared error

In [20]:
function quadraticloss(w,x,y)
    0.5 * sum(abs2, w * x - y)
end

quadraticloss (generic function with 1 method)

In [21]:
# (iter = 1048576, accuracy = 0.8498333333333333, wnorm = 1.2882874f0) in 79 secs with lr=0.0001
@time wadaline2 = optimize(quadraticloss,xtrn,ytrn,lr=0.0001);

(iter = 1, accuracy = 0.09736666666666667, wnorm = 0.0008820149f0)
(iter = 2, accuracy = 0.16698333333333334, wnorm = 0.0012901156f0)
(iter = 4, accuracy = 0.2466, wnorm = 0.0018553125f0)
(iter = 8, accuracy = 0.17425, wnorm = 0.00295244f0)
(iter = 16, accuracy = 0.23753333333333335, wnorm = 0.0044558235f0)
(iter = 32, accuracy = 0.20073333333333335, wnorm = 0.0076521044f0)
(iter = 64, accuracy = 0.32116666666666666, wnorm = 0.014028674f0)
(iter = 128, accuracy = 0.4724833333333333, wnorm = 0.025002351f0)
(iter = 256, accuracy = 0.5484333333333333, wnorm = 0.042062297f0)
(iter = 512, accuracy = 0.6185333333333334, wnorm = 0.06702255f0)
(iter = 1024, accuracy = 0.6920833333333334, wnorm = 0.105967544f0)
(iter = 2048, accuracy = 0.7441166666666666, wnorm = 0.16895463f0)
(iter = 4096, accuracy = 0.78585, wnorm = 0.25629073f0)
(iter = 8192, accuracy = 0.8055833333333333, wnorm = 0.3571566f0)
(iter = 16384, accuracy = 0.8308333333333333, wnorm = 0.4587624f0)
(iter = 32768, accuracy = 0.8391

### Softmax classifier maximizes the probabilities of correct answers
(or minimizes negative log likelihood, aka cross-entropy or softmax loss)

In [22]:
function negloglik(w,x,y)
    probs = exp.(w * x)
    probs = probs / sum(probs)
    class = argmax(y)
    -log(probs[class])
end

negloglik (generic function with 1 method)

In [24]:
# (iter = 1048576, accuracy = 0.9283833333333333, wnorm = 26.593485f0) in 120 secs with lr=0.01
@time wsoftmax2 = optimize(negloglik,xtrn,ytrn,lr=0.01);

(iter = 1, accuracy = 0.09863333333333334, wnorm = 0.09697416f0)
(iter = 2, accuracy = 0.1686, wnorm = 0.12180235f0)
(iter = 4, accuracy = 0.26025, wnorm = 0.15152907f0)
(iter = 8, accuracy = 0.25661666666666666, wnorm = 0.23889048f0)
(iter = 16, accuracy = 0.15921666666666667, wnorm = 0.37189257f0)
(iter = 32, accuracy = 0.2731166666666667, wnorm = 0.5208997f0)
(iter = 64, accuracy = 0.5370666666666667, wnorm = 0.77327734f0)
(iter = 128, accuracy = 0.6934833333333333, wnorm = 1.1921613f0)
(iter = 256, accuracy = 0.6787666666666666, wnorm = 1.8622694f0)
(iter = 512, accuracy = 0.7942333333333333, wnorm = 2.7099152f0)
(iter = 1024, accuracy = 0.79965, wnorm = 3.6680388f0)
(iter = 2048, accuracy = 0.8591666666666666, wnorm = 4.7135873f0)
(iter = 4096, accuracy = 0.8641166666666666, wnorm = 5.8873725f0)
(iter = 8192, accuracy = 0.8803833333333333, wnorm = 7.165706f0)
(iter = 16384, accuracy = 0.89935, wnorm = 8.757448f0)
(iter = 32768, accuracy = 0.9097166666666666, wnorm = 10.348306f0)
(